In [2]:
# 加载模块
import requests

from datetime import datetime
from pytz import timezone
from time import sleep

from vnpy.trader.object import BarData
from vnpy.trader.constant import Exchange, Interval

In [3]:
# 构建访问链接
base = "https://api.binance.com"
endpoint = "/api/v3/time"
url = base + endpoint

In [4]:
# 获取服务器时间
r = requests.get(url)
r.json()

{'serverTime': 1756612616629}

In [4]:
endpoint = "/api/v3/ticker/bookTicker"
url = base + endpoint

In [5]:
params = {"symbol": "BTCUSDT"}

In [6]:
r = requests.get(url, params=params)
r.json()

{'symbol': 'BTCUSDT',
 'bidPrice': '109222.46000000',
 'bidQty': '4.14224000',
 'askPrice': '109222.47000000',
 'askQty': '6.69309000'}

In [7]:
# 持续轮询输出
while True:
    r = requests.get(url, params=params)
    d = r.json()

    symbol = d["symbol"]
    bp = d["bidPrice"]
    bq = d["bidQty"]
    ap = d["askPrice"]
    aq = d["askQty"]

    msg = f"{symbol}. 买价{bp}[{bq}], 卖价{ap}[{aq}]"
    print(msg)
    sleep(1)

BTCUSDT. 买价109222.46000000[4.18795000], 卖价109222.47000000[6.69251000]
BTCUSDT. 买价109222.46000000[4.18800000], 卖价109222.47000000[5.79309000]
BTCUSDT. 买价109222.46000000[5.79888000], 卖价109222.47000000[3.48560000]
BTCUSDT. 买价109222.46000000[5.80585000], 卖价109222.47000000[3.96647000]


KeyboardInterrupt: 

In [12]:
# 这次来查询K线
endpoint = "/api/v3/klines"
url = base + endpoint

In [13]:
# 构建查询参数
params = {
    "symbol": "BTCUSDT",
    "interval": "1m"
}

In [14]:
# 发起K线查询请求
r = requests.get(url, params=params)
data = r.json()

In [15]:
# 查看数据类型
type(data)

list

In [16]:
# 查看具体数据
data[0]

[1756581480000,
 '108741.71000000',
 '108741.71000000',
 '108740.15000000',
 '108740.15000000',
 '0.92041000',
 1756581539999,
 '100086.39210130',
 226,
 '0.30279000',
 '32925.48024390',
 '0']

In [17]:
# 开放K线查询函数
CHINA_TZ = timezone("Asia/Shanghai")

def download_binance_minute_data(symbol: str):
    """基于代码和交易所下载数据"""
    base = "https://api.binance.com"
    endpoint = "/api/v3/klines"
    url = base + endpoint

    params = {
        "symbol": symbol,
        "interval": "1m"
    }

    r = requests.get(url, params=params)
    data = r.json()

    bar_data = []
    for l in data:
        dt = datetime.fromtimestamp(l[0]/1000)
        bar = BarData(
            symbol=symbol,
            exchange=Exchange.BINANCE,
            datetime=CHINA_TZ.localize(dt),
            interval=Interval.MINUTE,
            open_price=float(l[1]),
            high_price=float(l[2]),
            low_price=float(l[3]),
            close_price=float(l[4]),
            volume=float(l[5]),
            gateway_name="BINANCE"
        )
        bar_data.append(bar)
    return bar_data

In [19]:
# 测试一下结果
bar_data = download_binance_minute_data("BTCUSDT")
bar_data[0]

BarData(gateway_name='BINANCE', extra=None, symbol='BTCUSDT', exchange=<Exchange.BINANCE: 'BINANCE'>, datetime=datetime.datetime(2025, 8, 31, 3, 18, tzinfo=<DstTzInfo 'Asia/Shanghai' CST+8:00:00 STD>), interval=<Interval.MINUTE: '1m'>, volume=0.92041, turnover=0, open_interest=0, open_price=108741.71, high_price=108741.71, low_price=108740.15, close_price=108740.15)

In [22]:
import pandas as pd
df = pd.DataFrame.from_records([bar.__dict__ for bar in bar_data])

In [23]:
df

,gateway_name,symbol,exchange,datetime,interval,volume,turnover,open_interest,open_price,high_price,low_price,close_price,vt_symbol
0,BINANCE,BTCUSDT,Exchange.BINANCE,2025-08-31 03:18:00+08:00,Interval.MINUTE,0.92041,0,0,108741.71,108741.71,108740.15,108740.15,BTCUSDT.BINANCE
1,BINANCE,BTCUSDT,Exchange.BINANCE,2025-08-31 03:19:00+08:00,Interval.MINUTE,5.56694,0,0,108740.16,108769.30,108740.15,108769.30,BTCUSDT.BINANCE
2,BINANCE,BTCUSDT,Exchange.BINANCE,2025-08-31 03:20:00+08:00,Interval.MINUTE,0.19891,0,0,108769.30,108769.30,108769.29,108769.30,BTCUSDT.BINANCE
3,BINANCE,BTCUSDT,Exchange.BINANCE,2025-08-31 03:21:00+08:00,Interval.MINUTE,1.43955,0,0,108769.30,108769.30,108753.92,108753.92,BTCUSDT.BINANCE
4,BINANCE,BTCUSDT,Exchange.BINANCE,2025-08-31 03:22:00+08:00,Interval.MINUTE,0.94639,0,0,108753.93,108753.93,108736.50,108736.50,BTCUSDT.BINANCE
...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,BINANCE,BTCUSDT,Exchange.BINANCE,2025-08-31 11:33:00+08:00,Interval.MINUTE,0.90271,0,0,109179.59,109190.01,109179.58,109188.33,BTCUSDT.BINANCE
496,BINANCE,BTCUSDT,Exchange.BINANCE,2025-08-31 11:34:00+08:00,Interval.MINUTE,2.32748,0,0,109188.33,109213.63,109188.32,109213.63,BTCUSDT.BINANCE
497,BINANCE,BTCUSDT,Exchange.BINANCE,2025-08-31 11:35:00+08:00,Interval.MINUTE,5.75109,0,0,109213.62,109238.00,109213.62,109238.00,BTCUSDT.BINANCE
498,BINANCE,BTCUSDT,Exchange.BINANCE,2025-08-31 11:36:00+08:00,Interval.MINUTE,0.82804,0,0,109238.00,109238.00,109222.46,109222.46,BTCUSDT.BINANCE


In [24]:
# 保存到csv文件
df.to_csv("demo.csv")

In [25]:
from vnpy.trader.database import get_database
db = get_database()
db.save_bar_data(bar_data)

True

In [29]:
db.load_bar_data("BTCUSDT", Exchange.BINANCE, Interval.MINUTE, datetime(2025, 8, 31), datetime(2025, 8, 31))

[]